In [147]:
import time
import pandas as pd
import numpy as np

from functools import reduce

import matplotlib.pyplot as plt
%matplotlib inline

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options

In [5]:
## example headless driver

options = Options()
options.headless = True

browser = webdriver.Firefox(executable_path="./drivers/geckodriver", options=options)
browser.get('https://duckduckgo.com')

print('Title: %s' % browser.title)
browser.quit()

Title: DuckDuckGo — Privacy, simplified.


In [469]:
## functions to navigate nba.com/stats pages

def sort_by_name(browser):
    xpath_player_sort = '/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[2]/div[1]/table/thead/tr/th[2]'
    browser.find_element_by_xpath(xpath_player_sort).click()
    browser.find_element_by_xpath(xpath_player_sort).click()
    return

def select_all_pages(browser):
    xpath_page_selection = '/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select'
    xpath_all_pages = xpath_page_selection + '/option[1]'
    browser.find_element_by_xpath(xpath_all_pages).click()
    return

def select_per_100(browser):
    xpath_per_mode = '/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[3]/div/div/label/select'
    browser.find_element_by_xpath(xpath_per_mode).click()
    
    xpath_per_100 = '/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[3]/div/div/label/select/option[3]'
    browser.find_element_by_xpath(xpath_per_100).click()
    

def select_stat_type(browser, stat_type):
    """
    stat type mappings
    1: Traditional
    2: Advanced
    3: Misc
    4: Scoring
    5: Usage
    6: Opponent
    7: Defense
    """
    
    stat_table_dict = {'traditional': 1, 'advanced': 2, 'misc': 3, 'scoring': 4, 'usage': 5, 'opponent': 6, 'defense':7}
    n = stat_table_dict[stat_type]
    
    ## select the header to get drop down
    xpath_stat_type_button = '/html/body/main/div[2]/div/div[2]/div/nav-dropdown/nav/section[3]/div/a'
    browser.find_element_by_xpath(xpath_stat_type_button).click()

    ## navigate to different stat type
    xpath_stat_type = f'/html/body/main/div[2]/div/div[2]/div/nav-dropdown/nav/section[3]/ul/li[{n}]/a/span'
    browser.find_element_by_xpath(xpath_stat_type).click()
    
    if n in [1, 3, 6, 7]:
        time.sleep(2)
        select_per_100(browser)
    return

def select_season(browser, season):
    seasons_dict = dict(zip(range(2019, 1995, -1), [i for i in range(1, 100)]))
    n = seasons_dict[season]
    ## click 'SEASON' header
    xpath_season_header = '/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select'
    browser.find_element_by_xpath(xpath_season_header).click()
    ## select season
    xpath_select_season = f'/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select/option[{n}]'
    browser.find_element_by_xpath(xpath_select_season).click()
    return


def remove_equal_cols(df):
    "works with non-unique column names"
    dup_col_idxs = set()
    for i in range(len(df.columns)-1):
        if i in dup_col_idxs: continue
        for j in range(i+1, len(df.columns)):
            if df.iloc[:, i].equals(df.iloc[:, j]):                
                dup_col_idxs.add(j)
    df = df.iloc[:, list(set(range(len(df.columns)))-dup_col_idxs)]
    return df

In [319]:
list(df1.columns)

['A', 'B', 'C', 'D']

In [3]:
## Get df from raw table

def scrape_table(browser, stat_type: str):
    """
    Scrapes stats table present on browser and returns pandas DF. 
    
    1: Traditional
    2: Advanced
    3: Misc
    4: Scoring
    5: Usage
    6: Opponent
    7: Defense
    """
    stat_table_dict = {'traditional': 1, 'advanced': 2, 'misc': 3, 'scoring': 4, 'usage': 5, 'opponent': 6, 'defense':7}
    n = stat_table_dict[stat_type]
    
    ## read in raw table data
    xpath_stats_table = '/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[2]/div[1]'
    raw_table = browser.find_element_by_xpath(xpath_stats_table)
    table = raw_table.text.split('\n')
    
    ## columns for each table type...
    if n == 1:  # traditional
        cols = ['TEAM', 'AGE', 'GP', 'W', 'L', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP', 'DD2', 'TD3', '+/-']
    
    elif n == 2:  # advanced
        cols = ['TEAM', 'AGE', 'GP', 'W', 'L', 'MIN', 'OFFRTG', 'DEFRTG', 'NETRTG', 'AST%', 'AST/TO', 'AST RATIO', 'OREB%', 'DREB%', 'REB%', 'TO RATIO', 'EFG%', 'TS%', 'USG%', 'PACE', 'PIE']
    
    elif n == 3: # misc
        cols = ['TEAM','AGE','GP','W','L','MIN','PTS OFF TO','2ND PTS','FBPS','PITP','OPP PTS OFF TO','OPP 2ND PTS','OPP FBPS','OPP PITP','BLK','BLKA','PF','PFD']

    elif n == 4: # scoring
        cols = ['TEAM', 'AGE', 'GP', 'W','L','MIN','%FGA 2PT','%FGA 3PT','%PTS 2PT','%PTS 2PT MR','%PTS 3PT','%PTS FBPS','%PTS FT','%PTS OFFTO','%PTS PITP','2FGM %AST','2FGM %UAST','3FGM %AST','3FGM %UAST','FGM %AST','FGM %UAST']
    
    elif n == 5:  # usage
        cols = ['TEAM', 'AGE', 'GP', 'W', 'L', 'MIN', 'USG%', '%FGM', '%FGA', '%3PM', '%3PA', '%FTM', '%FTA', '%OREB', '%DREB', '%REB', '%AST', '%TOV', '%STL', '%BLK', '%BLKA', '%PF', '%PFD', '%PTS']
    
#     ## MISSING DATA FROM PRIOR 2010'S
#     elif table_num == 6: # opponent
#         cols = ['TEAM','GP','W','L','MIN','OPP FGM','OPP FGA','OPP FG%','OPP 3PM','OPP 3PA','OPP 3P%','OPP FTM','OPP FTA','OPP FT%','OPP OREB','OPP DREB','OPP REB','OPP AST','OPP TOV','OPP STL','OPP BLK','OPP BLKA','OPP PF','OPP PFD','OPP PTS','+/-',]
    
    elif n == 7:  # defense
        cols = ['TEAM','AGE','GP','W','L','MIN','DEF RTG','DREB','DREB%','%DREB','STL','STL%','BLK','%BLK','OPP PTS OFF TOV','OPP PTS 2ND CHANCE','OPP PTS FB','OPP PTS PAINT','DEF WS']

    ## extract position of first player row
    for i, line in enumerate(table):
        if line and line[0] == 'A' and len(line.split(' ')) == 2:
            body = table[i:] 
            break
            
    ## parse body of table now and extract index, stats
    index, all_stats = [], []
    for i in range(0, len(body), 2):
        player = body[i]
        if len(player.split(' ')) > 0 and len(player.split(' ')) < 3:
            index.append(player)
            stats = body[i+1].split(' ')
            all_stats.append(stats)
        
    return pd.DataFrame(all_stats, index=index, columns=cols)

In [470]:
def scrape_combine_season_data(season: int, stat_types: list):
    
    ## set up browser
    generic_nba_stats_url = 'https://stats.nba.com/players/traditional/?sort=PTS&dir=-1'
    browser = webdriver.Firefox(executable_path="./drivers/geckodriver")
    browser.get(generic_nba_stats_url)
    time.sleep(3)  # ensure loading...
    
    dfs = []
    for stat_type in stat_types:
        select_stat_type(browser, stat_type); time.sleep(3)
        select_season(browser, season); time.sleep(3)
        sort_by_name(browser); time.sleep(3)
        select_all_pages(browser); time.sleep(3)
        df = scrape_table(browser, stat_type)
        dfs.append(df)
        time.sleep(3)
     
    ## merge all dataframes together on index and only use unique col names
    combined_stats = reduce(lambda left, right: pd.merge(left, right[right.columns.difference(left.columns)], left_index=True, right_index=True), dfs)
    ## remove the remaining duplicate columns with slightly diff names (i.e. 'DEFRTG' and 'DEF RTG')
    combined_stats = remove_equal_cols(combined_stats)
    
    ## ADD YEAR SUFFIX TO INDEX I.E. (JAMES HARDEN 19, MICHAEL JORDAN 98)
    
    return combined_stats

In [474]:
## TESTING
stat_types = ['traditional', 'defense', 'usage', 'advanced', 'misc', 'scoring']
# stat_types = ['traditional', 'advanced']
stats_12 = scrape_combine_season_data(2012, stat_types)

NoSuchElementException: Message: Unable to locate element: /html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[2]/div[1]/table/thead/tr/th[2]


In [467]:
stats_01.shape

(436, 86)

In [471]:
stats = remove_equal_cols(stats_01)
stats.shape

(436, 86)

In [473]:
set(stats_01.columns) - set(merge_cols)

{'%3PA',
 '%3PM',
 '%AST',
 '%BLKA',
 '%FGA',
 '%FGM',
 '%FTA',
 '%FTM',
 '%OREB',
 '%PF',
 '%PFD',
 '%PTS',
 '%REB',
 '%TOV'}

In [457]:
## TESTING

concat_df = pd.concat(dfs, axis=1)
concat_cols = concat_df.columns


merge_df = reduce(lambda left, right: pd.merge(left, right[right.columns.difference(left.columns)], left_index=True, right_index=True, ), dfs)
merge_cols = merge_df.columns

print(len(concat_cols))
print(len(merge_cols))

print('-----')

concat_df = remove_identical_cols(concat_df)
concat_cols = concat_df.columns

merge_df = remove_identical_cols(merge_df)
merge_cols = merge_df.columns

print(len(concat_cols))
print(len(merge_cols))

print('----')

extra_cols = set(merge_cols) - set(concat_cols)
print(len(extra_cols))

## There are 2 different MIN variations (shown by double MIN col in concat_cols which are filtered out in the reduce merge)

107
77
-----
73
72
----
0


In [459]:
seen = []
for col in concat_cols:
    if col not in seen:
        seen.append(col)
    else:
        print('dup:', col)
        

dup: MIN


In [461]:
concat_cols.sort_values()

Index(['%BLK', '%DREB', '%FGA 2PT', '%FGA 3PT', '%PTS 2PT', '%PTS 2PT MR',
       '%PTS 3PT', '%PTS FBPS', '%PTS FT', '%PTS OFFTO', '%PTS PITP', '+/-',
       '2FGM %AST', '2FGM %UAST', '2ND PTS', '3FGM %AST', '3FGM %UAST', '3P%',
       '3PA', '3PM', 'AGE', 'AST', 'AST RATIO', 'AST%', 'AST/TO', 'BLK',
       'BLKA', 'DD2', 'DEF RTG', 'DEF WS', 'DREB', 'DREB%', 'EFG%', 'FBPS',
       'FG%', 'FGA', 'FGM', 'FGM %AST', 'FGM %UAST', 'FP', 'FT%', 'FTA', 'FTM',
       'GP', 'L', 'MIN', 'MIN', 'NETRTG', 'OFFRTG', 'OPP PTS 2ND CHANCE',
       'OPP PTS FB', 'OPP PTS OFF TOV', 'OPP PTS PAINT', 'OREB', 'OREB%',
       'PACE', 'PF', 'PFD', 'PIE', 'PITP', 'PTS', 'PTS OFF TO', 'REB', 'REB%',
       'STL', 'STL%', 'TD3', 'TEAM', 'TO RATIO', 'TOV', 'TS%', 'USG%', 'W'],
      dtype='object')

In [453]:
extra_cols = set(concat_cols) - set(merge_cols)
print(len(extra_cols))

0


In [430]:
for i in list(concat_cols.sort_values()): print(i)

%BLK
%DREB
%FGA 2PT
%FGA 3PT
%PTS 2PT
%PTS 2PT MR
%PTS 3PT
%PTS FBPS
%PTS FT
%PTS OFFTO
%PTS PITP
+/-
2FGM %AST
2FGM %UAST
2ND PTS
3FGM %AST
3FGM %UAST
3P%
3PA
3PM
AST
AST RATIO
AST%
AST/TO
BLKA
DD2
DEF RTG
DEF WS
EFG%
FBPS
FG%
FGA
FGM
FGM %AST
FGM %UAST
FP
FT%
FTA
FTM
NETRTG
OFFRTG
OPP PTS 2ND CHANCE
OPP PTS FB
OPP PTS OFF TOV
OPP PTS PAINT
OREB
OREB%
PACE
PFD
PIE
PITP
PTS
PTS OFF TO
REB
REB%
STL%
TD3
TO RATIO
TOV
TS%
USG%


,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,...,%PTS FBPS,%PTS FT,%PTS OFFTO,%PTS PITP,2FGM %AST,2FGM %UAST,3FGM %AST,3FGM %UAST,FGM %AST,FGM %UAST
Aaron McKie,6.5,2.6,6.1,43.0,0.9,2.7,32.3,0.4,0.7,62.5,...,15.8,6.6,20.4,25.0,73.2,26.8,95.0,5.0,80.3,19.7
Aaron Williams,11.9,4.7,10.3,46.0,0.0,0.0,0.0,2.5,2.8,88.2,...,8.2,20.5,17.8,63.0,82.8,17.2,0.0,0.0,82.8,17.2
Adonal Foyle,10.5,4.7,9.3,50.2,0.0,0.0,0.0,1.2,2.1,55.6,...,6.3,11.4,12.5,79.5,64.1,35.9,0.0,0.0,64.1,35.9
Adrian Griffin,11.4,4.4,12.2,36.0,0.2,0.7,22.2,2.5,3.3,75.0,...,12.6,21.9,23.2,33.1,51.8,48.2,100,0.0,53.4,46.6
Al Harrington,23.7,9.3,20.2,45.9,0.3,1.5,21.6,4.8,7.2,67.2,...,11.6,20.4,18.0,53.5,45.3,54.7,93.8,6.3,47.0,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zaza Pachulia,17.2,6.0,13.3,45.2,0.0,0.0,0.0,5.2,7.0,74.6,...,6.6,30.1,15.7,60.7,66.9,33.1,0.0,0.0,66.9,33.1
Zeljko Rebraca,19.3,7.6,13.3,56.8,0.0,0.0,0.0,4.2,4.8,85.9,...,2.7,21.5,11.8,59.6,66.9,33.1,0.0,0.0,66.9,33.1
Zendon Hamilton,16.3,3.5,10.2,34.4,0.0,0.0,0.0,9.3,15.3,60.4,...,7.8,56.9,23.5,35.3,81.8,18.2,0.0,0.0,81.8,18.2
Zoran Planinic,22.5,8.1,18.0,44.8,1.6,4.1,37.5,4.8,6.8,69.7,...,18.0,21.2,21.2,41.5,44.4,55.6,93.3,6.7,53.8,46.2


In [364]:
extra_cols

{'AGE', 'BLK', 'DREB', 'DREB%', 'GP', 'L', 'MIN', 'PF', 'STL', 'TEAM', 'W'}

In [356]:
stats = pd.concat(dfs, axis=1)
print(stats.shape)

concat_dups, stats = remove_identical_cols(stats)
len(concat_dups)

print(stats.shape)

# stats = reduce(lambda left, right: pd.merge(left, right[right.columns.difference(left.columns)], left_index=True, right_index=True, ), dfs)
# print(stats.shape)

# stats = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True, ), dfs)
# print(stats.shape)

# stats = remove_identical_cols(stats)
# print(stats.shape)



    

(460, 107)
(460, 61)


In [ ]:
# there are 107 total cols
# 30 exact name repeats

# 72 'unique' cols according to merge verison and 61 according to concat
# 16 col discpepancy

In [353]:
stats = pd.concat(dfs, axis=1)
print(stats.shape)

stats = remove_identical_cols(stats)
print(stats.shape)

(460, 107)
['TEAM', 'TEAM', 'TEAM', 'TEAM', 'AGE', 'AGE', 'AGE', 'AGE', 'GP', 'GP', 'GP', 'GP', 'W', 'W', 'W', 'W', 'L', 'L', 'L', 'L', 'MIN', 'MIN', 'MIN', 'MIN', 'DREB', 'STL', 'BLK', 'BLK', 'PF', 'TEAM', 'TEAM', 'TEAM', 'AGE', 'AGE', 'AGE', 'GP', 'GP', 'GP', 'W', 'W', 'W', 'L', 'L', 'L', 'MIN', 'MIN', 'MIN', 'DEFRTG', 'DREB%', 'BLK', 'OPP PTS OFF TO', 'OPP 2ND PTS', 'OPP FBPS', 'OPP PITP', 'TEAM', 'TEAM', 'AGE', 'AGE', 'GP', 'GP', 'W', 'W', 'L', 'L', 'MIN', 'MIN', 'TEAM', 'AGE', 'GP', 'W', 'L', 'MIN']
(460, 61)


In [352]:
stats = pd.concat(dfs, axis=1)
cols = list(stats.columns.sort_values())
print(stats.shape); print()
seen = []
dups = {}
tot = 0
for col in cols:
    if col not in seen:
        seen.append(col)
    else:
        tot += 1
        if col not in dups:
            dups[col]=1
            
        else:
            dups[col] += 1
            
            
print(tot)
print()
dups
        
        

(460, 107)

30



{'AGE': 4,
 'BLK': 2,
 'DREB': 1,
 'DREB%': 1,
 'GP': 4,
 'L': 4,
 'MIN': 4,
 'PF': 1,
 'STL': 1,
 'TEAM': 4,
 'W': 4}

In [346]:
a = {'a':1}
'a' in a

True

In [334]:
# stats = reduce(lambda left, right: pd.merge(left, right[right.columns.difference(left.columns)], left_index=True, right_index=True, ), dfs)
stats = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True, ), dfs)

cols = list(stats.columns.sort_values())
print(len(cols)); print()
for col in cols: print(col)

107

%BLK
%DREB
%FGA 2PT
%FGA 3PT
%PTS 2PT
%PTS 2PT MR
%PTS 3PT
%PTS FBPS
%PTS FT
%PTS OFFTO
%PTS PITP
+/-
2FGM %AST
2FGM %UAST
2ND PTS
3FGM %AST
3FGM %UAST
3P%
3PA
3PM
AGE
AGE_x
AGE_x
AGE_y
AGE_y
AST
AST RATIO
AST%
AST/TO
BLK
BLKA
BLK_x
BLK_y
DD2
DEF RTG
DEF WS
DEFRTG
DREB%_x
DREB%_y
DREB_x
DREB_y
EFG%
FBPS
FG%
FGA
FGM
FGM %AST
FGM %UAST
FP
FT%
FTA
FTM
GP
GP_x
GP_x
GP_y
GP_y
L
L_x
L_x
L_y
L_y
MIN
MIN_x
MIN_x
MIN_y
MIN_y
NETRTG
OFFRTG
OPP 2ND PTS
OPP FBPS
OPP PITP
OPP PTS 2ND CHANCE
OPP PTS FB
OPP PTS OFF TO
OPP PTS OFF TOV
OPP PTS PAINT
OREB
OREB%
PACE
PFD
PF_x
PF_y
PIE
PITP
PTS
PTS OFF TO
REB
REB%
STL%
STL_x
STL_y
TD3
TEAM
TEAM_x
TEAM_x
TEAM_y
TEAM_y
TO RATIO
TOV
TS%
USG%
W
W_x
W_x
W_y
W_y


In [300]:

a = df1.copy()
a.drop(['A', 'A', 'A'], axis=1)

,B,C,D
55,B0,C0,D0
65,B1,C1,D1
75,B2,C2,D2
85,B3,C3,D3


In [295]:
combined_stats = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True, ), dfs)
combined_stats.shape

dups = []
for i, col1 in enumerate(combined_stats.columns):
    for col2 in combined_stats.columns[i+1:]:
        if combined_stats[col1].equals(combined_stats[col2]):
            dups.append(col2)
combined_stats.drop(dups, inplace=True, axis=1)
combined_stats.shape

(460, 72)

In [290]:
combined_stats = reduce(lambda left, right: pd.merge(left, right[right.columns.difference(left.columns)], left_index=True, right_index=True, ), dfs)

In [289]:
# dups = [col for i, col in enumerate(combined_stats.columns) if combined_stats[col].equals(combined_stats[col2] for col2 in combined_stats.columns[i+1:])]

dups = []
for i, col1 in enumerate(combined_stats.columns):
    for col2 in combined_stats.columns[i+1:]:
        if combined_stats[col1].equals(combined_stats[col2]):
            print('MATCH:', col1, col2)
            dups.append(col2)
dups
    

MATCH: DEF RTG DEFRTG
MATCH: OPP PTS 2ND CHANCE OPP 2ND PTS
MATCH: OPP PTS FB OPP FBPS
MATCH: OPP PTS OFF TOV OPP PTS OFF TO
MATCH: OPP PTS PAINT OPP PITP


['DEFRTG', 'OPP 2ND PTS', 'OPP FBPS', 'OPP PTS OFF TO', 'OPP PITP']

In [272]:
combo = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), dfs)
combo.shape

(460, 107)

In [273]:
combined_stats = reduce(lambda left, right: pd.merge(left, right[right.columns.difference(left.columns)], left_index=True, right_index=True), dfs)
combined_stats.shape

(460, 77)

In [276]:
# for i in combined_stats.columns.sort_values(): print(i)
    
for i, col1 in enumerate(combined_stats.columns):
    print('\n----------', col1)
    for col2 in combined_stats.columns[i+1:]:
        if combined_stats[col1].equals(combined_stats[col2]):
            print('MATCH WITH:', col2)


---------- TEAM

---------- AGE

---------- GP

---------- W

---------- L

---------- MIN

---------- PTS

---------- FGM

---------- FGA

---------- FG%

---------- 3PM

---------- 3PA

---------- 3P%

---------- FTM

---------- FTA

---------- FT%

---------- OREB

---------- DREB

---------- REB

---------- AST

---------- TOV

---------- STL

---------- BLK

---------- PF

---------- FP

---------- DD2

---------- TD3

---------- +/-

---------- %BLK

---------- %DREB

---------- DEF RTG
MATCH WITH: DEFRTG

---------- DEF WS

---------- DREB%

---------- OPP PTS 2ND CHANCE
MATCH WITH: OPP 2ND PTS

---------- OPP PTS FB
MATCH WITH: OPP FBPS

---------- OPP PTS OFF TOV
MATCH WITH: OPP PTS OFF TO

---------- OPP PTS PAINT
MATCH WITH: OPP PITP

---------- STL%

---------- AST RATIO

---------- AST%

---------- AST/TO

---------- DEFRTG

---------- EFG%

---------- NETRTG

---------- OFFRTG

---------- OREB%

---------- PACE

---------- PIE

---------- REB%

---------- TO RATIO

-----

In [261]:
for df in dfs:
    print(df.head())

               TEAM AGE  GP   W   L   MIN   PTS  FGM   FGA   FG%  ...   REB  \
Aaron McKie     PHI  32  68  35  33  48.5   6.5  2.6   6.1  43.0  ...   7.4   
Aaron Williams  TOR  33  42  13  29  50.9  11.9  4.7  10.3  46.0  ...   9.8   
Adonal Foyle    GSW  30  78  31  47  50.7  10.5  4.7   9.3  50.2  ...  12.8   
Adrian Griffin  CHI  30  69  38  31  49.9  11.4  4.4  12.2  36.0  ...  11.1   
Al Harrington   ATL  25  66  10  56  52.0  23.7  9.3  20.2  45.9  ...   9.4   

                AST  TOV  STL  BLK    PF    FP DD2 TD3    +/-  
Aaron McKie     4.4  1.3  2.1  0.7   5.0  29.2   0   0    0.5  
Aaron Williams  1.3  2.8  0.8  1.3  11.4  29.2   0   0  -22.2  
Adonal Foyle    1.7  1.7  0.8  4.7   6.0  43.2   3   0   -0.8  
Adrian Griffin  4.0  2.3  3.3  0.3   5.9  39.1   0   0    3.0  
Al Harrington   4.3  4.2  1.7  0.3   5.1  43.4  12   0  -10.5  

[5 rows x 28 columns]
               TEAM AGE  GP   W   L   MIN DEF RTG DREB DREB% %DREB  STL  STL%  \
Aaron McKie     PHI  32  68  35  33  

In [259]:
for i, col1 in enumerate(combo.columns):
    print('\n----------', col1)
    for col2 in combo.columns[i:]:
        if combo[col1].equals(combo[col2]):
            print('MATCH WITH:', col2)




---------- TEAM_x
MATCH WITH: TEAM_x
MATCH WITH: TEAM_x

---------- AGE_x
MATCH WITH: AGE_x
MATCH WITH: AGE_x

---------- GP_x
MATCH WITH: GP_x
MATCH WITH: GP_x

---------- W_x
MATCH WITH: W_x
MATCH WITH: W_x

---------- L_x
MATCH WITH: L_x
MATCH WITH: L_x

---------- MIN_x
MATCH WITH: MIN_x
MATCH WITH: MIN_x

---------- PTS
MATCH WITH: PTS

---------- FGM
MATCH WITH: FGM

---------- FGA
MATCH WITH: FGA

---------- FG%
MATCH WITH: FG%

---------- 3PM
MATCH WITH: 3PM

---------- 3PA
MATCH WITH: 3PA

---------- 3P%
MATCH WITH: 3P%

---------- FTM
MATCH WITH: FTM

---------- FTA
MATCH WITH: FTA

---------- FT%
MATCH WITH: FT%

---------- OREB
MATCH WITH: OREB

---------- DREB_x
MATCH WITH: DREB_x
MATCH WITH: DREB_y

---------- REB
MATCH WITH: REB

---------- AST
MATCH WITH: AST

---------- TOV
MATCH WITH: TOV

---------- STL_x
MATCH WITH: STL_x
MATCH WITH: STL_y

---------- BLK_x
MATCH WITH: BLK_x
MATCH WITH: BLK_y
MATCH WITH: BLK

---------- PF_x
MATCH WITH: PF_x
MATCH WITH: PF_y

-----

In [177]:
## TESTING

stat_types = ['traditional', 'defense', 'advanced', 'misc', 'scoring']
# stat_types = ['traditional', 'advanced']
res = scrape_combine_season_data(2004, stat_types)

In [242]:
## investigating duplicate column names

stat_types = ['traditional', 'defense', 'advanced', 'misc', 'scoring']
dups = ['TEAM', 'AGE', 'GP', 'W', 'L', 'MIN']
dfs = [res[k] for k in res]
for i, (df, stat_type) in enumerate(zip(dfs, stat_types), 1):
    print('\n-------------------', stat_type, '---------------------')
    
    for j, other in enumerate(dfs[i:]):
        print('\n---------other:', stat_types[i+j])
        for col1 in df.columns:
            if col1 not in dups:
                for col2 in other.columns:
                    if df[col1].equals(other[col2]):
#                     if col1 == col2:
                        print('MATCH:', col1, col2)
        
        
        
        
        
#         for col2 in dfs[i+1].columns:
#             if col1 not in dups:
#                 if col1 == col2:
#                     print('\nMATCHING COL')
#                     print(col1, 'and', col2)


------------------- traditional ---------------------

---------other: defense
MATCH: DREB DREB
MATCH: STL STL
MATCH: BLK BLK

---------other: advanced

---------other: misc
MATCH: BLK BLK
MATCH: PF PF

---------other: scoring

------------------- defense ---------------------

---------other: advanced
MATCH: DEF RTG DEFRTG
MATCH: DREB% DREB%

---------other: misc
MATCH: BLK BLK
MATCH: OPP PTS OFF TOV OPP PTS OFF TO
MATCH: OPP PTS 2ND CHANCE OPP 2ND PTS
MATCH: OPP PTS FB OPP FBPS
MATCH: OPP PTS PAINT OPP PITP

---------other: scoring

------------------- advanced ---------------------

---------other: misc

---------other: scoring

------------------- misc ---------------------

---------other: scoring

------------------- scoring ---------------------


In [244]:
## function that takes in df and

combo

,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,AST RATIO,OREB%,DREB%,REB%,TO RATIO,EFG%,TS%,USG%,PACE,PIE
Aaron McKie,PHI,32,68,35,33,48.5,6.5,2.6,6.1,43.0,...,36.4,1.7,13.5,7.5,11.0,50.0,51.0,6.8,99.06,6.6
Aaron Williams,TOR,33,42,13,29,50.9,11.9,4.7,10.3,46.0,...,8.4,5.5,15.2,10.1,17.9,46.0,51.8,12.8,95.28,2.0
Adonal Foyle,GSW,30,78,31,47,50.7,10.5,4.7,9.3,50.2,...,12.3,9.3,15.2,12.2,12.7,50.2,51.4,10.4,94.87,7.2
Adrian Griffin,CHI,30,69,38,31,49.9,11.4,4.4,12.2,36.0,...,20.0,7.2,13.8,10.4,11.3,36.6,41.9,13.6,96.64,9.0
Al Harrington,ATL,25,66,10,56,52.0,23.7,9.3,20.2,45.9,...,13.4,5.7,13.9,9.6,13.1,46.8,50.8,23.6,92.35,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zaza Pachulia,MIL,21,74,27,47,52.5,17.2,6.0,13.3,45.2,...,10.6,10.1,18.9,14.5,12.3,45.2,52.6,16.6,91.50,9.6
Zeljko Rebraca,LAC,33,58,27,31,53.0,19.3,7.6,13.3,56.8,...,7.5,5.8,14.3,10.3,14.1,56.8,62.5,16.0,90.38,10.2
Zendon Hamilton,MIL,30,16,7,9,51.4,16.3,3.5,10.2,34.4,...,8.2,11.9,16.8,14.3,16.4,34.4,48.0,18.9,93.37,3.5
Zoran Planinic,NJN,22,43,20,23,53.0,22.5,8.1,18.0,44.8,...,15.3,3.8,10.2,7.0,14.3,49.1,53.4,21.9,90.86,8.8


In [211]:
for i, df, stat_type in enumerate(dfs) stat_types:
    print(i)

ValueError: too many values to unpack (expected 3)

In [212]:
for i, df, stat_type in enumerate(zip(dfs, stat_types)):
    print(stat_type)

ValueError: not enough values to unpack (expected 3, got 2)

In [216]:
aa = [1,1,1]
bb = [2,2,2]
for i, (a, b) in enumerate(zip(aa, bb)):
    print(a)

1
1
1


In [201]:
a = (zip(enumerate(dfs), stat_types))
for x in a:
    print(type(x))
    print(x)
    print('-------------')

<class 'tuple'>
((0,                    TEAM AGE  GP   W   L   MIN   PTS  FGM   FGA   FG%  ...  \
Aaron McKie         PHI  32  68  35  33  48.5   6.5  2.6   6.1  43.0  ...   
Aaron Williams      TOR  33  42  13  29  50.9  11.9  4.7  10.3  46.0  ...   
Adonal Foyle        GSW  30  78  31  47  50.7  10.5  4.7   9.3  50.2  ...   
Adrian Griffin      CHI  30  69  38  31  49.9  11.4  4.4  12.2  36.0  ...   
Al Harrington       ATL  25  66  10  56  52.0  23.7  9.3  20.2  45.9  ...   
...                 ...  ..  ..  ..  ..   ...   ...  ...   ...   ...  ...   
Zaza Pachulia       MIL  21  74  27  47  52.5  17.2  6.0  13.3  45.2  ...   
Zeljko Rebraca      LAC  33  58  27  31  53.0  19.3  7.6  13.3  56.8  ...   
Zendon Hamilton     MIL  30  16   7   9  51.4  16.3  3.5  10.2  34.4  ...   
Zoran Planinic      NJN  22  43  20  23  53.0  22.5  8.1  18.0  44.8  ...   
Zydrunas Ilgauskas  CLE  30  78  40  38  53.0  26.7  9.3  19.8  46.8  ...   

                     REB  AST  TOV  STL  BLK    PF    

In [183]:
## checking the repeating columsn to make sure their are no overlapping names for diff stats

# dfs = [res[k] for k in res]
for i, stat_type in enumerate(stat_types):
    if i < len(dfs):
        for col1 in res[stat_type].columns:
            for col2 in res[stat_types[i+1]].columns:
                if res[col1].equals(res[col2]):
                    print('EQUAL COLS')
                    print(f'{col1} and {col2}\n')

    

KeyError: 'TEAM'

In [137]:
res.keys()

dict_keys(['traditional', 'defense', 'advanced', 'misc', 'scoring'])

In [143]:
for k in res:
    print(len(res[k].index))

460
460
460
460
460


In [149]:
dfs = [res[k] for k in res]
len(dfs)

5

In [153]:
tot_cols = sum([len(res[k].columns) for k in res])
print(tot_cols)



107
28
19
21
18
21


In [162]:
a = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), dfs)
# a[]

In [ ]:
cols_to_use = df2.columns.difference(df.columns)

b = 

In [172]:
b = reduce(lambda left, right: pd.merge(left, right[right.columns.difference(left.columns)], left_index=True, right_index=True), dfs)

In [174]:
for col in b.columns: print(col)

TEAM
AGE
GP
W
L
MIN
PTS
FGM
FGA
FG%
3PM
3PA
3P%
FTM
FTA
FT%
OREB
DREB
REB
AST
TOV
STL
BLK
PF
FP
DD2
TD3
+/-
%BLK
%DREB
DEF RTG
DEF WS
DREB%
OPP PTS 2ND CHANCE
OPP PTS FB
OPP PTS OFF TOV
OPP PTS PAINT
STL%
AST RATIO
AST%
AST/TO
DEFRTG
EFG%
NETRTG
OFFRTG
OREB%
PACE
PIE
REB%
TO RATIO
TS%
USG%
2ND PTS
BLKA
FBPS
OPP 2ND PTS
OPP FBPS
OPP PITP
OPP PTS OFF TO
PFD
PITP
PTS OFF TO
%FGA 2PT
%FGA 3PT
%PTS 2PT
%PTS 2PT MR
%PTS 3PT
%PTS FBPS
%PTS FT
%PTS OFFTO
%PTS PITP
2FGM %AST
2FGM %UAST
3FGM %AST
3FGM %UAST
FGM %AST
FGM %UAST


In [163]:
for col in a.columns:
    print(col)

TEAM_x
AGE_x
GP_x
W_x
L_x
MIN_x
PTS
FGM
FGA
FG%
3PM
3PA
3P%
FTM
FTA
FT%
OREB
DREB_x
REB
AST
TOV
STL_x
BLK_x
PF_x
FP
DD2
TD3
+/-
TEAM_y
AGE_y
GP_y
W_y
L_y
MIN_y
DEF RTG
DREB_y
DREB%_x
%DREB
STL_y
STL%
BLK_y
%BLK
OPP PTS OFF TOV
OPP PTS 2ND CHANCE
OPP PTS FB
OPP PTS PAINT
DEF WS
TEAM_x
AGE_x
GP_x
W_x
L_x
MIN_x
OFFRTG
DEFRTG
NETRTG
AST%
AST/TO
AST RATIO
OREB%
DREB%_y
REB%
TO RATIO
EFG%
TS%
USG%
PACE
PIE
TEAM_y
AGE_y
GP_y
W_y
L_y
MIN_y
PTS OFF TO
2ND PTS
FBPS
PITP
OPP PTS OFF TO
OPP 2ND PTS
OPP FBPS
OPP PITP
BLK
BLKA
PF_y
PFD
TEAM
AGE
GP
W
L
MIN
%FGA 2PT
%FGA 3PT
%PTS 2PT
%PTS 2PT MR
%PTS 3PT
%PTS FBPS
%PTS FT
%PTS OFFTO
%PTS PITP
2FGM %AST
2FGM %UAST
3FGM %AST
3FGM %UAST
FGM %AST
FGM %UAST


In [14]:
tr_idx = tr.index
ad_idx = ad.index

tr.index.equals(ad_idx)

True

In [16]:
combined = pd.concat([tr, ad], axis=1)
combined.head()

,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,AST RATIO,OREB%,DREB%,REB%,TO RATIO,EFG%,TS%,USG%,PACE,PIE
Aaron McKie,PHI,32,68,35,33,48.5,6.5,2.6,6.1,43.0,...,36.4,1.7,13.5,7.5,11.0,50.0,51.0,6.8,99.06,6.6
Aaron Williams,TOR,33,42,13,29,50.9,11.9,4.7,10.3,46.0,...,8.4,5.5,15.2,10.1,17.9,46.0,51.8,12.8,95.28,2.0
Adonal Foyle,GSW,30,78,31,47,50.7,10.5,4.7,9.3,50.2,...,12.3,9.3,15.2,12.2,12.7,50.2,51.4,10.4,94.87,7.2
Adrian Griffin,CHI,30,69,38,31,49.9,11.4,4.4,12.2,36.0,...,20.0,7.2,13.8,10.4,11.3,36.6,41.9,13.6,96.64,9.0
Al Harrington,ATL,25,66,10,56,52.0,23.7,9.3,20.2,45.9,...,13.4,5.7,13.9,9.6,13.1,46.8,50.8,23.6,92.35,9.5


In [22]:
print(len(tr.columns))
print(len(ad.columns))

28
21


In [133]:
## seeing how to concat the stat tables from same season

df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']},
                        index=[55, 65, 75, 85])

df2 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3'],
                           'E': ['E0', 'E1', 'E2', 'E3']},
                        index=[55, 65, 75, 85])


# combined = pd.concat([df1, df2], axis=1)

# combined = combined.loc[:, combined.columns.unique()]
# combined = pd.merge(df1, df2, how='left')
# combined = df1.join(df2, how=index)
# combined = pd.merge(df1, df2, left_index=True, right_index=True)
combined = pd.merge(df1, df2, how='right')
combined = pd.merge(df1, df2)
combined



combined = pd.merge(df1, df2, left_index=True, right_index=True)
combined

,A_x,B_x,C_x,D_x,A_y,B_y,C_y,D_y,E
55,A0,B0,C0,D0,A0,B0,C0,D0,E0
65,A1,B1,C1,D1,A1,B1,C1,D1,E1
75,A2,B2,C2,D2,A2,B2,C2,D2,E2
85,A3,B3,C3,D3,A3,B3,C3,D3,E3


In [135]:
combined['A_x'].equals(combined['A_y'])

True

In [66]:
combined.iloc[:, 0].equals(combined.iloc[:, 4])

True

In [63]:
print(combined.columns.duplicated())
print(combined.columns)
print(~combined.columns.duplicated())

[False False False False  True  True  True  True False]
Index(['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'E'], dtype='object')
[ True  True  True  True False False False False  True]


In [90]:
## Dropping duplicate columns
a = combo.loc[:, ~combo.columns.duplicated()]

In [92]:
a.columns

Index(['TEAM', 'AGE', 'GP', 'W', 'L', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM',
       '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV',
       'STL', 'BLK', 'PF', 'FP', 'DD2', 'TD3', '+/-', 'OFFRTG', 'DEFRTG',
       'NETRTG', 'AST%', 'AST/TO', 'AST RATIO', 'OREB%', 'DREB%', 'REB%',
       'TO RATIO', 'EFG%', 'TS%', 'USG%', 'PACE', 'PIE'],
      dtype='object')

In [6]:
len(res['traditional'])
len(res['advanced'])

tr = res['traditional']
ad = res['advanced']

print(tr.head())
ad.head()

               TEAM AGE  GP   W   L   MIN   PTS  FGM   FGA   FG%  ...   REB  \
Aaron McKie     PHI  32  68  35  33  48.5   6.5  2.6   6.1  43.0  ...   7.4   
Aaron Williams  TOR  33  42  13  29  50.9  11.9  4.7  10.3  46.0  ...   9.8   
Adonal Foyle    GSW  30  78  31  47  50.7  10.5  4.7   9.3  50.2  ...  12.8   
Adrian Griffin  CHI  30  69  38  31  49.9  11.4  4.4  12.2  36.0  ...  11.1   
Al Harrington   ATL  25  66  10  56  52.0  23.7  9.3  20.2  45.9  ...   9.4   

                AST  TOV  STL  BLK    PF    FP DD2 TD3    +/-  
Aaron McKie     4.4  1.3  2.1  0.7   5.0  29.2   0   0    0.5  
Aaron Williams  1.3  2.8  0.8  1.3  11.4  29.2   0   0  -22.2  
Adonal Foyle    1.7  1.7  0.8  4.7   6.0  43.2   3   0   -0.8  
Adrian Griffin  4.0  2.3  3.3  0.3   5.9  39.1   0   0    3.0  
Al Harrington   4.3  4.2  1.7  0.3   5.1  43.4  12   0  -10.5  

[5 rows x 28 columns]


,TEAM,AGE,GP,W,L,MIN,OFFRTG,DEFRTG,NETRTG,AST%,...,AST RATIO,OREB%,DREB%,REB%,TO RATIO,EFG%,TS%,USG%,PACE,PIE
Aaron McKie,PHI,32,68,35,33,16.6,100.4,99.7,0.7,13.5,...,36.4,1.7,13.5,7.5,11.0,50.0,51.0,6.8,99.06,6.6
Aaron Williams,TOR,33,42,13,29,7.4,86.4,106.4,-20.0,4.9,...,8.4,5.5,15.2,10.1,17.9,46.0,51.8,12.8,95.28,2.0
Adonal Foyle,GSW,30,78,31,47,21.8,102.3,102.4,-0.2,4.8,...,12.3,9.3,15.2,12.2,12.7,50.2,51.4,10.4,94.87,7.2
Adrian Griffin,CHI,30,69,38,31,9.5,94.8,91.1,3.8,13.4,...,20.0,7.2,13.8,10.4,11.3,36.6,41.9,13.6,96.64,9.0
Al Harrington,ATL,25,66,10,56,38.6,97.8,108.0,-10.2,14.8,...,13.4,5.7,13.9,9.6,13.1,46.8,50.8,23.6,92.35,9.5
